In [75]:
import sqlalchemy as sq
import geopandas as gpd  # type: ignore
import pandas as pd  # type: ignore
import numpy as np
from dotenv import load_dotenv
import os, sys, calendar

sys.path.append("../")
from Shared.DataService import DataService  # type: ignore

In [76]:
MIN_MONTH = 1
MAX_MONTH = 12

MIN_YEAR = 1995
MAX_YEAR = 2022
TABLENAME = "agg_dly_weather"

load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [77]:
db = DataService(PG_DB, PG_ADDR, PG_PORT, PG_USER, PG_PW)
conn = db.connect()

In [78]:
# pulling weather station data from the database
weatherDataQuery = sq.text(
    """
    SELECT * FROM public.ab_station_data
    UNION
    SELECT * FROM public.mb_station_data
    UNION
    SELECT * FROM public.sk_station_data;
    """
)

weatherData = pd.read_sql(weatherDataQuery, conn)

In [79]:
# pulling station data from the database
stationDataQuery = sq.text(
    """
    SELECT station_id, district FROM public.stations_dly
    WHERE district IS NOT NULL;
    """
)

stationData = pd.read_sql(stationDataQuery, conn)
stationData[['district']] = stationData[['district']].astype(int)

In [80]:
# merge both the weather station data and the station data together
df = weatherData.merge(stationData, on="station_id")

In [81]:
df

,station_id,date,year,month,day,max_temp,min_temp,mean_temp,total_rain,total_snow,total_precip,snow_on_grnd,district
0,3010010,2002-11-26,2002,11,26,5.1,-13.2,-4.1,0.0,0.0,0.0,0.0,4860
1,3010010,2002-11-27,2002,11,27,7.6,-2.8,2.4,0.0,0.0,0.0,0.0,4860
2,3010010,2002-11-28,2002,11,28,8.7,1.7,5.2,0.0,0.0,0.0,0.0,4860
3,3010010,2002-11-29,2002,11,29,3.5,-3.6,-0.1,0.0,0.0,0.0,0.0,4860
4,3010010,2002-11-30,2002,11,30,7.8,-6.3,0.8,0.0,0.0,0.0,0.0,4860
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387796,506B047,2005-09-04,2005,9,4,21.1,11.6,16.4,1.0,0.0,1.0,0.0,4612
3387797,506B047,2005-09-05,2005,9,5,22.3,9.3,15.8,9.5,0.0,9.5,0.0,4612
3387798,506B047,2005-09-06,2005,9,6,16.1,11.4,13.8,0.0,0.0,0.0,0.0,4612
3387799,506B047,2005-09-07,2005,9,7,17.8,9.6,13.7,0.0,0.0,0.0,0.0,4612


In [82]:
# aggregate the values in the dataframe by date and district
agg_df = df.groupby(["district", "date"]).agg({
    "max_temp": "mean",
    "min_temp": "mean",
    "mean_temp": "mean",
    "total_rain": ["min", "max", "mean"],
    "total_snow": ["min", "max", "mean"],
    "total_precip": ["min", "max", "mean"],
    "snow_on_grnd": ["min", "max", "mean"]
}).reset_index()

# sets the column names for the aggregate dataframe
agg_df.columns = [  # type: ignore
    "district",
    "date",
    "max_temp",
    "min_temp",
    "mean_temp",
    "min_total_rain",
    "max_total_rain",
    "mean_total_rain",
    "min_total_snow",
    "max_total_snow",
    "mean_total_snow",
    "min_total_precip",
    "max_total_precip",
    "mean_total_precip",
    "min_snow_on_grnd",
    "max_snow_on_grnd",
    "mean_snow_on_grnd"
]

In [83]:
agg_df

,district,date,max_temp,min_temp,mean_temp,min_total_rain,max_total_rain,mean_total_rain,min_total_snow,max_total_snow,mean_total_snow,min_total_precip,max_total_precip,mean_total_precip,min_snow_on_grnd,max_snow_on_grnd,mean_snow_on_grnd
0,4601,1992-12-01,2.000000,-6.200000,-2.100000,0.0,0.0,0.0,0.0,0.0,0.000000,1.1,1.1,1.100000,0.0,0.0,0.000000
1,4601,1992-12-02,-5.900000,-15.700000,-10.800000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,4601,1992-12-05,-3.000000,-16.000000,-9.500000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
3,4601,1992-12-06,-7.500000,-20.600000,-14.100000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
4,4601,1992-12-07,-9.500000,-19.400000,-14.500000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921681,4870,2022-12-27,-16.286667,-20.503333,-18.380000,0.0,0.0,0.0,0.0,4.0,0.246667,0.0,7.2,2.946667,0.0,37.0,6.033333
921682,4870,2022-12-28,-13.950000,-19.213333,-16.590000,0.0,0.0,0.0,0.0,0.4,0.013333,0.0,4.0,0.553333,0.0,37.0,6.800000
921683,4870,2022-12-29,-11.080000,-20.066667,-15.566667,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.8,0.123333,0.0,36.0,6.666667
921684,4870,2022-12-30,-9.356667,-18.296667,-13.830000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.3,0.010000,0.0,36.0,6.533333


In [84]:
# figure out the date range for date processing - puts all 365 days as MO-DA into dates
dates = [] # all 365 days as MO-DA - strings

months = [str(month) for month in range(MIN_MONTH, MAX_MONTH + 1)]  # the month range we want to pull data from - strings
for month in months:
    if len(month) == 1:
        month = '0' + month
    
    numDays = calendar.monthrange(2001, int(month))[1]
    days = [str(day) for day in range(1, numDays + 1)]

    for day in days:
        if len(day) == 1:
            day = '0' + day
            
        dates.append(f'{month}-{day}')

In [85]:
years = [year for year in range(MIN_YEAR, MAX_YEAR + 1)]  # the year range we want to pull data from - ints
uniqueDistricts = stationData['district'].unique()

cols = agg_df.columns
cols = cols.drop('district')
cols = cols.drop('date')

In [86]:
listForDF = [] # loads all data where each row is its own dictionary so that it may added to a dataframe later (fast processing)

for year in years:
    print(f'Processing year: {year}')
    
    for district in uniqueDistricts:
        currData = {} # for each year/district combination create a dictionary

         # adds the year and district
        currData['year'] = year
        currData['district'] = district

        for date in dates: # for each day we want to grab all attributes and establish them as columns i.e MO-DA:attribute 
            fullDate = np.datetime64(f'{str(year)}-{date}') # calculates the date we are current processing
            currRow = agg_df.loc[(agg_df["date"] == fullDate) & (agg_df["district"] == district)] # grab the row from the aggregated df

            for col in cols: # parse each of the desired columns
                currAttr = f'{date}:{col}' # the current attribute which corresponds to the date and the column
                currVal = 0 # defaults as zero incase it does not exist

                if len(currRow[col]) == 1:
                    currVal = currRow[col].item() # the current value from the loaded data
                
                currData[currAttr] = currVal

        listForDF.append(currData)

Processing year: 1995
Processing year: 1996
Processing year: 1997
Processing year: 1998
Processing year: 1999
Processing year: 2000
Processing year: 2001
Processing year: 2002
Processing year: 2003
Processing year: 2004
Processing year: 2005
Processing year: 2006
Processing year: 2007
Processing year: 2008
Processing year: 2009
Processing year: 2010
Processing year: 2011
Processing year: 2012
Processing year: 2013
Processing year: 2014
Processing year: 2015
Processing year: 2016
Processing year: 2017
Processing year: 2018
Processing year: 2019
Processing year: 2020
Processing year: 2021
Processing year: 2022


In [87]:
final_df = pd.DataFrame(listForDF)

In [88]:
final_df

,year,district,01-01:max_temp,01-01:min_temp,01-01:mean_temp,01-01:min_total_rain,01-01:max_total_rain,01-01:mean_total_rain,01-01:min_total_snow,01-01:max_total_snow,...,12-31:mean_total_rain,12-31:min_total_snow,12-31:max_total_snow,12-31:mean_total_snow,12-31:min_total_precip,12-31:max_total_precip,12-31:mean_total_precip,12-31:min_snow_on_grnd,12-31:max_snow_on_grnd,12-31:mean_snow_on_grnd
0,1995,4860,-8.0375,-20.200000,-14.137500,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,7.3,1.70,0.0,1.5,0.444444,0.0,36.0,16.333333
1,1995,4810,-7.4200,-17.000000,-12.220000,0.0,0.0,0.0,0.0,0.5,...,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,19.0,6.600000
2,1995,4840,-8.3000,-19.425000,-13.875000,0.0,0.0,0.0,0.0,4.2,...,0.0,0.0,2.9,1.52,0.0,7.7,2.280000,0.0,24.0,8.800000
3,1995,4830,-5.7500,-20.258333,-13.033333,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,5.1,0.392308,0.0,28.0,5.615385
4,1995,4731,-7.3000,-19.650000,-13.500000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,2022,4605,-28.3000,-33.000000,-30.700000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,27.0,27.0,27.000000
1060,2022,4720,-24.8000,-36.900000,-30.850000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.1,0.050000,23.0,34.0,28.500000
1061,2022,4611,-25.8000,-35.600000,-30.700000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.1,0.1,0.100000,14.0,14.0,14.000000
1062,2022,4751,-22.9000,-33.600000,-28.300000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,6.0,6.0,6.000000


In [89]:
final_df.to_sql(TABLENAME, conn, schema="public", if_exists="append", index=False)

OperationalError: (psycopg2.errors.TooManyColumns) tables can have at most 1600 columns

[SQL: 
CREATE TABLE public.agg_dly_weather (
	year BIGINT, 
	district INTEGER, 
	"01-01:max_temp" FLOAT(53), 
	"01-01:min_temp" FLOAT(53), 
	"01-01:mean_temp" FLOAT(53), 
	"01-01:min_total_rain" FLOAT(53), 
	"01-01:max_total_rain" FLOAT(53), 
	"01-01:mean_total_rain" FLOAT(53), 
	"01-01:min_total_snow" FLOAT(53), 
	"01-01:max_total_snow" FLOAT(53), 
	"01-01:mean_total_snow" FLOAT(53), 
	"01-01:min_total_precip" FLOAT(53), 
	"01-01:max_total_precip" FLOAT(53), 
	"01-01:mean_total_precip" FLOAT(53), 
	"01-01:min_snow_on_grnd" FLOAT(53), 
	"01-01:max_snow_on_grnd" FLOAT(53), 
	"01-01:mean_snow_on_grnd" FLOAT(53), 
	"01-02:max_temp" FLOAT(53), 
	"01-02:min_temp" FLOAT(53), 
	"01-02:mean_temp" FLOAT(53), 
	"01-02:min_total_rain" FLOAT(53), 
	"01-02:max_total_rain" FLOAT(53), 
	"01-02:mean_total_rain" FLOAT(53), 
	"01-02:min_total_snow" FLOAT(53), 
	"01-02:max_total_snow" FLOAT(53), 
	"01-02:mean_total_s

In [ ]:
db.cleanup()